Hasta ahora hemos utilizado la transferencia de aprendizaje. 
Deep Learning es excelente a la hora de poder construir rápidamente modelos precisos, incluso con relativamente pocos datos, pero el aprendizaje de transferencia no es necesario en datos grandes  y solo funciona cuando su caso de uso tiene patrones visuales similares a los datos utilizados en el modelo preentrenado, por ejemplo: un modelo preentrenado basado en las fotos cotidianas non funcionará bien para aplicaciones de fotografías de satelites o para fotos medicas de una maquina de resonancia, asi que si quieres ser versatil con profundidad hay que aprender a construir modelos.

Cargamos los datos (fotos) del nido M para almacenar todas las imágenes en una sola:
**train_file = "--.train.csv"**
**raw_data = pd.read_csv(train_file)**
Cada imagen se representa como una fila del CSV, la primera columna tiene una etiqueta en estos datos, la etiqueta dice qué digito se muestra en cada fila del CSV. 
El resto de columnas representan las intensidades de los pixeles.

Preparación de los datos para extraer etiquetas y remodelas la intensidad de los píxeles:

**img_rows, img_cols = 28,28** #cuadricula
**num_classes = 10**

def data_prep(raw):
    out_y = **keras.utils.to_categorical(raw.label, mum_classes)** #función categórica suministramos la etiqueta de fila variable objetivo, así como el número de valores diferentes (num-classes), que devuelve 10 valores objetivo con 10 columnas binarias separadas

**num_images = raw.shape[0]**

x_as_array = **raw.values[:,1:]** #intesidades de pixeles , nos da los datos como una matriz numpy, usamos indexación para tomar todo después de la primera columna (xq la primera columna era la etiqueta) 

x_shaped_array = **x_as_array.reshape(num_images, img_rows,img_cols,1)** #cambiamos la forma de los datos a una matriz 4d que está indexado por el número de imagen, número de fila, número de columna, y como esta en escala de grises y no en color solo habrá 1 canal .

out_x = **x_shaped_array / 255** #divide los valores entre 255 para que todos los datos estén entre 0 y 1, esto mejora la optimización con los parámetros predeterminados. 

**return out_x, out_y**

Para el optimizador de átomos, esta función nos dio una serie de predictores que llamaré X y una matriz del objetivo que llamaré Y y construiremos nuestro modelo:

**x,y = data_prep(raw_data)**

En este modelo destacamos que la primera capa es convolucional 2d:
**model.add(Conv2D(20,kernel_size = (3,3), activation='relu,input_shape=(img_rows,img_cols,1)))**

También agregamos una capa convolucional antes del aplanamiento:
**model.add(Conv2D(20,kernel_size=(3,3),activation='relu))**

Argumentos de estas capas convolucionales:
Número de circunvoluciones o filtros que se incluirán en esa capa.
El tamaño y pixeles de las convoluciones que se llama tamaño del nucleo.
Funcion de activacion: para la primera capa tengo que especificar la forma de cada imagen (shape), pero no para las otras capas.

En este caso se usaron 2 capas convolucionales, pero se podrian haber usado más.

Capa aplanada: 
**model.add(Flatten())** para convertir las salidas de las capas anteriores en una representacion 1s para cada imagen.

Los modelos generalmente funcionan mejor si se agrega una capa densa entre los planos y una capa de predicción final, así que agregamos una capa densa con 128 nodos:
**model.add(Dense(128, activation='relu'))**

Capa de salida:
**model.add(Dense(num_classes, activation='softmax'))**

Luego compilamos ( **función compile**) y lo ajustamos (**funcion fit**) en ella proporciono los predictores x y el objetivo, especifico el tamaño del lote (**batch_size**) y varias epocas (**epochs**) y finalmente la puntuación de validación (**validation_split=0.2**), para decir que el 20% de los datos deben reservarse para la validación, dejando el 80% para la capacitación.

Al ejecutar esto, puedo ver la precisión de 10% entrenando aproximadamente el 98% bastante rápido,.

Si aumentamos el numero de capas o el numero de convoluciones en una capa, aumentas la capacidad del modelo para adaptarse a los datos de entrenamiento. 


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout


img_rows, img_cols = 28, 28
num_classes = 10

def data_prep(raw):
    out_y = keras.utils.to_categorical(raw.label, num_classes)

    num_images = raw.shape[0]
    x_as_array = raw.values[:,1:]
    x_shaped_array = x_as_array.reshape(num_images, img_rows, img_cols, 1)
    out_x = x_shaped_array / 255
    return out_x, out_y

train_file = "../input/digit-recognizer/train.csv"
raw_data = pd.read_csv(train_file)

x, y = data_prep(raw_data)
model = Sequential()
model.add(Conv2D(20, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows, img_cols, 1)))
model.add(Conv2D(20, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x, y,
          batch_size=128,
          epochs=2,
          validation_split = 0.2)


**EJERCICIO**
Has visto cómo construir un modelo desde cero para identificar dígitos escritos a mano. Ahora construirá un modelo para identificar diferentes tipos de ropa. Para hacer modelos que entrenan rápidamente, trabajaremos con imágenes muy pequeñas (de baja resolución).

Como ejemplo, su modelo tomará imágenes como esta y lo identificará como un zapato:


**PREPARACIÓN DE LOS DATOS**

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras

img_rows, img_cols = 28, 28
num_classes = 10

def prep_data(raw):
    y = raw[:, 0]
    out_y = keras.utils.to_categorical(y, num_classes)
    
    x = raw[:,1:]
    num_images = raw.shape[0]
    out_x = x.reshape(num_images, img_rows, img_cols, 1)
    out_x = out_x / 255
    return out_x, out_y

fashion_file = "../input/fashionmnist/fashion-mnist_train.csv"
fashion_data = np.loadtxt(fashion_file, skiprows=1, delimiter=',')
x, y = prep_data(fashion_data)

# Set up code checking
from learntools.core import binder
binder.bind(globals())
from learntools.deep_learning.exercise_7 import *
print("Setup Complete")

**1)START THE MODEL**
Create a `Sequential` model called `fashion_model`. Don't add layers yet.


In [ ]:
fashion_model = Sequential()

**2) Add the first layer**
Agregue la primera capa Conv2D a fashion_model. Debe tener 12 filtros, un kernel_size de 3 y la función de activación relu. La primera capa siempre requiere que especifique input_shape. Hemos guardado el número de filas y columnas en las variables img_rows e img_cols respectivamente, por lo que la forma de entrada en este caso es (img_rows, img_cols, 1).

In [ ]:
fashion_model.add(Conv2D(12, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows, img_cols, 1)))

**3) Agregue las capas restantes**
Agregue 2 más convolucionales (capas Conv2D) con 20 filtros cada uno, activación 'relu' y un tamaño de núcleo de 3. Siga eso con una capa Flatten, y luego una capa Densa con 100 neuronas.

Agregue su capa de predicción a fashion_model. Esta es una capa densa. Ya tenemos una variable llamada num_classes. Use esta variable cuando especifique el número de nodos en esta capa. La activación debe ser softmax (o tendrá problemas más adelante).

In [ ]:
# Your code here
fashion_model.add(Conv2D(20, kernel_size=(3, 3), activation='relu'))
fashion_model.add(Conv2D(20, kernel_size=(3, 3), activation='relu'))
fashion_model.add(Flatten())
fashion_model.add(Dense(100, activation='relu'))
fashion_model.add(Dense(num_classes, activation='softmax'))
# Check your answer
q_3.check()

**COMPILAMOS EL MODELO**

In [ ]:
# Your code to compile the model in this cell
fashion_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

**AJUSTAMOS EL MODELO**

In [ ]:
fashion_model.fit(x, y,
          batch_size=100,
          epochs=4,
          validation_split = 0.2)